In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Tải vncorenlp

In [ ]:
!pip3 install vncorenlp
%mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
%mv VnCoreNLP-1.1.1.jar vncorenlp/
%mv vi-vocab vncorenlp/models/wordsegmenter/
%mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [ ]:
import os
import csv
from tqdm import tqdm
from transformers import AutoTokenizer
from vncorenlp import VnCoreNLP

# Tokenize và trích xuất thực thể

## Hàm tokenize và trích xuất thực thể

In [ ]:
def get_data(tsv_file_path, output_entities_extraction):
    rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx500m')

    X = []
    y = []

    with open(tsv_file_path, 'r', encoding='utf-8') as f:
        tsv_reader = csv.reader(f, delimiter='\t')
        rows = []

        for row in tqdm(tsv_reader):
            if len(row) >= 1:
                content = row[0]
                label = row[1]

                if not content.strip():
                    print("Skipping empty line")
                    continue

                try:
                    content = content.encode('utf-8', errors='replace').decode('utf-8')
                    annotated_text = rdrsegmenter.annotate(content)

                    entities = []
                    tokenized_text = []
                    for sentence in annotated_text['sentences']:
                        for word in sentence:
                            tokenized_text.append(word['form'])
                            if word['nerLabel'] != 'O':  # Not a non-entity
                                entities.append(word['form'])

                    keywords = ','.join(entities)
                    tokenized_content = ' '.join(tokenized_text)

                    X.append(tokenized_content)
                    y.append(label)

                    rows.append({'content': content, 'keywords': keywords})
                except Exception as e:
                    print(f"Error processing text: {content}")
                    print(f"Error: {e}")
                    continue

    with open(output_entities_extraction, 'w', encoding='utf-8', newline='') as f:
      writer = csv.DictWriter(f, fieldnames=['content', 'keywords'], delimiter='\t')
      writer.writeheader()
      writer.writerows(rows)


train_path = '/content/drive/MyDrive/Data Mining Field-oriented/data/raw/articles_training.tsv'
output_entities_extraction = '/content/drive/MyDrive/Data Mining Field-oriented/output/output_ner.tsv'
X_data, y_data = get_data(train_path)

24963it [11:02, 37.66it/s]


## Kiểm chứng kết quả

In [ ]:
print(X_data[8892])
print(y_data[8892])

Trước đó , ngày 6/9 , Cơ_quan Cảnh_sát điều_tra , Công_an TP Hồ_Chí_Minh khởi_tố , bắt tạm giam 2 bảo_mẫu gồm : Diệp_Ngọc_Tuyền ( 47 tuổi , quê Sóc_Trăng ) và Nguyễn_Thị_Ngọc_Cẩm ( 46 tuổi , quê Đồng_Nai ) cùng về tội " Hành_hạ người khác " . Trong thời_gian qua , Cơ_quan Cảnh_sát điều_tra , Công_an TP Hồ_Chí_Minh thường_xuyên mời bà Giáp_Thị_Sông_Hương cùng một_số bảo_mẫu , nhân_viên của mái ấm Hoa_Hồng lên làm_việc . Để phục_vụ công_tác mở_rộng điều_tra , xử_lý các đối_tượng , Phòng Cảnh_sát hình_sự - Công_an TP Hồ_Chí_Minh phát đi thông_báo tìm người đã nhận con_nuôi hoặc người gửi con cho Mái ấm Hoa_Hồng , Quận 12 , TP Hồ_Chí_Minh hoặc bà Giáp_Thị_Sông_Hương , các nhà hảo_tâm đã ủng_hộ tiền , vật_chất cho mái ấm này để phục_vụ điều_tra cung_cấp thông_tin làm rõ . Trong lần phát_biểu trước_đây , bà Võ_Thị_Chính , Phó Chủ_tịch UBND Quận 12 cho biết : chủ cơ_sở có sự đối_phó rất tinh_vi với cơ_quan quản_lý_Nhà_nước , cơ_sở chỉ được phép nuôi 39 trẻ nhưng sau khi báo_chí phản_ánh , các

## Xuất output

In [ ]:
from google.colab import files

output_path = '/content/drive/MyDrive/Data Mining Field-oriented/consolidated/tokenized_articles_training.tsv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w', encoding='utf-8', newline='') as out_file:
    writer = csv.writer(out_file, delimiter='\t')
    for x, y in zip(X_data, y_data):
        writer.writerow([x, y])

files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>